## Methodische & Digitale Kompetenz (DIKO) - Modularbeit Eigenleistung - SS 2024 - Maximilian Fiebig

#### Installation von OpenAI-Bibliothek (https://pypi.org/project/openai/)

In [1]:
pip install openai python-dotenv

Note: you may need to restart the kernel to use updated packages.


#### Erstellung der Azure OpenAI-Instanz (https://learn.microsoft.com/de-de/azure/ai-services/openai/how-to/switching-endpoints#api-key)
- Der API KEY ist ähnlich wie ein Passwort
- Man speichert ihn in einer externen .env-Datei, damit niemand den Key im Code sehen kann
- Die API-Version ist die genaue Spezifikation/Anleitung, damit die Bibliothek genau weiß, wie das JSON formatiert werden muss. Sie ändert sich durch Updates, deswegen muss sie mit angegeben werden. Die Version 2024-02-01 ist die von MUCGPT genutze Version bei Azure. (https://learn.microsoft.com/en-gb/azure/ai-services/openai/api-version-deprecation#latest-ga-api-release)

In [2]:
from dotenv import load_dotenv
from openai import AzureOpenAI

load_dotenv() # Laden der externen .env-Datei, in der API-Key und Azure Endpoint gespeichert sind.

client = AzureOpenAI(
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),  
    api_version="2024-02-01",
    azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT")
)

Chat Completions API-Aufruf: https://platform.openai.com/docs/guides/text-generation/chat-completions-api

- Temperature: Im Prompting bestimmt die Temperatur die Gewichtung von Kreativität in den Antworten der KI. Sie kann auch die Konsistenz und Vorhersehbarkeit der KI-Antworten beeinflussen. Eine niedrigere Temperatur führt dazu, dass die KI eher Wörter wählt, die wahrscheinlicher sind, und die Antworten beim gleichen Prompt sich ähnlich sind. Näheres dazu wird in der MUCGPT_temperature.ipynb Datei erläutert.

In [3]:
response = client.chat.completions.create(
  model="chat", # 'Chat' ist das zugehörige Deployment bei Azure, dahinter liegt gpt-3.5-turbo.
  messages=[
    {"role": "system", "content": "Du bist ein Experte für Baustoffkunde."}, # Anweisung an das Sprachmodell, optional
    {"role": "user", "content": "Was sind die Bestandteile von Beton?"} # Frage von Nutzer an das Sprachmodell
  ], 
  temperature=1 # Steuerung der Kreativität
)

In [4]:
print(response.choices[0].message.content) # Ausgabe der Antwort des Chatmodells

Beton besteht aus Zement, Gesteinskörnungen wie Sand und Kies, Wasser und gegebenenfalls Zusatzstoffen wie zum Beispiel Betonzusatzmitteln oder Zusätzen zur Verbesserung der Verarbeitbarkeit oder der Festigkeit. Der Zement dient als Bindemittel und reagiert chemisch mit dem Wasser, wodurch eine feste und stabile Struktur entsteht. Die Gesteinskörnungen dienen als Füllstoffe und geben dem Beton Festigkeit. Zusatzstoffe können je nach Anforderung des Bauprojekts hinzugefügt werden.


#### Importieren der .txt-Datei mit Fragen

In [5]:
with open("Fragen.txt") as datei:
    fragen = datei.readlines() # In der txt.-Datei sollen alle Zeilen gelesen werden.

print(fragen) # Ausgabe der Fragen

['Wie viel wiegt 1m³ Beton?\n', 'Gibt es klimaneutrale Baustoffe?\n', 'Welche Eigenschaften hat Asphalt?\n', 'Wie wird ein Ziegelstein hergestellt?\n', 'Kann man eine Straße aus Beton bauen?\n', 'Aus welchem Baustoff bestehen Brücken?\n', 'Wie sieht die Rezyklierbarkeit von Beton aus?\n', 'Ist Stahl ein guter Baustoff für U-Bahngleise?\n', 'Ist es möglich, ein Haus aus dem Baustoff Holz zu bauen?\n', 'Welches Material wird für den Einsatz in Hochbauten eingesetzt?']


Formatierung der Ausgabe

In [6]:
fragen = [line.rstrip() for line in fragen] # .rstrip entfernt Zeilenumbürche (/n), https://statistikguru.de/python/python-zeilenweise-lesen.html
print(fragen)

['Wie viel wiegt 1m³ Beton?', 'Gibt es klimaneutrale Baustoffe?', 'Welche Eigenschaften hat Asphalt?', 'Wie wird ein Ziegelstein hergestellt?', 'Kann man eine Straße aus Beton bauen?', 'Aus welchem Baustoff bestehen Brücken?', 'Wie sieht die Rezyklierbarkeit von Beton aus?', 'Ist Stahl ein guter Baustoff für U-Bahngleise?', 'Ist es möglich, ein Haus aus dem Baustoff Holz zu bauen?', 'Welches Material wird für den Einsatz in Hochbauten eingesetzt?']


#### Erstellen von Funktion und Liste zum auslesen der Fragen und Antworten

In [7]:
def frage_gpt(text, temp=0): # Definieren der Funktion frage_gpt und Übergabe der Eingabewerte. 'text'fungiert als Platzhalter, Temperature wird auf 0 festgelegt.
    response = client.chat.completions.create(
        model="chat",
        messages=[
            {"role": "system", "content": "Du bist ein Experte für Baustoffkunde."}, # Amweisung an das Sprachmodell, das dieses ein Experte für Baustoffkunde ist.
            {"role": "user", "content": text}
        ], 
        temperature=temp
    )
    return response.choices[0].message.content

In [8]:
import os
os.remove('Ausgabe_Allg.txt') # Öffnen der Datei Ausgabe_Allg.txt, um die Fragen und Antworten in dieser Datei auszugeben.

antworten = [] # Erstellen einer Liste für die Ausgabe der Antworten.

file = open('Ausgabe_Allg.txt', 'a')
for frage in fragen: # Schleife, um alle Fragen aus der Liste abzuarbeiten und die entsprechende Antwort des Chatmodells mit abzufragen.
    antwort = frage_gpt(frage)
    antworten.append(antwort) # Liste mit Antworten auffüllen, und speichern im Dataframe. Wichtig für Ausgabe als Excel-Datei.
    output_text = "Frage: {}\nAntwort: {}\n\n".format(frage, antwort) # Formatierung der Antwortausgabe.
    file.write(output_text) # Übergabe der Antwort an die .txt-Ausgabedatei.
    print(output_text)

file.close()

Frage: Wie viel wiegt 1m³ Beton?
Antwort: Das Gewicht von 1m³ Beton hängt von der Dichte des Betons ab, die wiederum von den verwendeten Materialien und deren Mischungsverhältnis abhängt. In der Regel wiegt 1m³ Normalbeton (mit einer Dichte von etwa 2,3 t/m³) etwa 2.300 kg oder 2,3 Tonnen. Allerdings kann das Gewicht je nach Art des Betons und der verwendeten Zuschlagsstoffe variieren.


Frage: Gibt es klimaneutrale Baustoffe?
Antwort: Ja, es gibt Baustoffe, die als klimaneutral gelten können. Ein Beispiel dafür sind Baustoffe aus nachwachsenden Rohstoffen wie Holz, Stroh oder Hanf. Diese Materialien binden während ihres Wachstums CO2 und setzen es bei der Verbrennung oder dem Verrotten wieder frei. Wenn sie als Baustoffe verwendet werden, bleibt das CO2 langfristig gebunden und trägt somit nicht zum Klimawandel bei.

Auch mineralische Baustoffe wie Beton oder Ziegel können klimaneutral sein, wenn sie aus recycelten Materialien hergestellt werden oder wenn bei ihrer Herstellung CO2-neu

#### Installation der Bibliothek Pandas und Ausgabe als Excel-Datei

In [9]:
pip install pandas

Note: you may need to restart the kernel to use updated packages.


In [10]:
import pandas as pd # Ausgabe der Werte im Tabellenformat: https://pandas.pydata.org/docs/getting_started/intro_tutorials/01_table_oriented.html#min-tut-01-tableoriented

df = pd.DataFrame(
    { 
        "Frage": fragen,
        "Antwort": antworten
    }
)
df # Tatsächlicher Ausgabebegriff

,Frage,Antwort
0,Wie viel wiegt 1m³ Beton?,Das Gewicht von 1m³ Beton hängt von der Dichte...
1,Gibt es klimaneutrale Baustoffe?,"Ja, es gibt Baustoffe, die als klimaneutral ge..."
2,Welche Eigenschaften hat Asphalt?,"Asphalt ist ein viskoelastischer Baustoff, der..."
3,Wie wird ein Ziegelstein hergestellt?,Ein Ziegelstein wird aus Ton hergestellt. Der ...
4,Kann man eine Straße aus Beton bauen?,"Ja, Straßen aus Beton sind eine gängige Bauwei..."
5,Aus welchem Baustoff bestehen Brücken?,Brücken können aus verschiedenen Baustoffen ge...
6,Wie sieht die Rezyklierbarkeit von Beton aus?,"Beton ist ein Baustoff, der aus Zement, Sand, ..."
7,Ist Stahl ein guter Baustoff für U-Bahngleise?,"Ja, Stahl ist ein sehr guter Baustoff für U-Ba..."
8,"Ist es möglich, ein Haus aus dem Baustoff Holz...","Ja, es ist möglich, ein Haus aus Holz zu bauen..."
9,Welches Material wird für den Einsatz in Hochb...,"Es gibt viele Materialien, die für den Einsatz..."


In [11]:
df.to_excel("F&A.xlsx") # Ausgabe der Tabelle im Excel-Format.